# Library import

In [1]:
import pandas as pd
import numpy as np
import torch
import os

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
from tqdm import tqdm

d:\College's course\Social Media Insight\IE403.P21-NLPmodel\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Load

In [1]:
# Load in cleaned dataset
df = pd.read_csv('Data/Cleaned.csv')

NameError: name 'pd' is not defined

In [2]:
path = os.path.join(os.getcwd(), 'Data', 'DataTestComment.csv')
df_test = pd.read_csv(path)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_24640\17680083.py:2: DtypeWarning: Columns (14,15,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(path)


# Annotation

## Calling twitter-ROBERTA-sentimental-model

In [3]:
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [4]:
labels = ["negative", "neutral", "positive"]

def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = softmax(logits, dim=-1)
    pred_class = torch.argmax(probs).item()
    return labels[pred_class]

In [5]:
for idx, text in enumerate(df_test["commentBody"]):
    try:
        tokenizer(text)
    except Exception as e:
        print(f"Row {idx}: error - {e} | type: {type(text)} | value: {text}")

Row 178898: error - text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples). | type: <class 'float'> | value: nan
Row 231286: error - text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples). | type: <class 'float'> | value: nan


In [7]:
print(df_test.loc[[178898, 231286], ["commentBody"]])

       commentBody
178898         NaN
231286         NaN


In [9]:
df_test = df_test.dropna(subset=["commentBody"])               # removes NaNs
df_test = df_test[df_test["commentBody"].str.len() > 0]           # removes empty strings

In [6]:
tqdm.pandas(desc="Processing")
df_test['sentiment'] = df_test['commentBody'].progress_apply(get_sentiment)

Processing:  77%|███████▋  | 178899/231449 [4:20:55<1:16:38, 11.43it/s]


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).